In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/')

Mounted at /content/drive


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00


In [ ]:
# -*- coding: utf-8 -*-
# Modified for Kaggle: Batch Job Recommendations & Total Experience Calculation

# Install necessary packages
import pandas as pd
import numpy as np
import re
import spacy
import os
from datetime import datetime
from PyPDF2 import PdfReader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")



In [ ]:
# Check if the pickle file exists
if os.path.exists("job_df.pkl"):
    job_df = pd.read_pickle("job_df.pkl")
    print("DataFrame loaded from pickle file.")
else:
    # Load the dataset from Kaggle's "Input" section
  #  csv_file_path = "/kaggle/input/upwork-job-postings-dataset-2024-50k-records/upwork-jobs.csv"  # Replace with your dataset path
    job_df = pd.read_csv("./input/upwork-jobs.csv")  # Replace with your dataset path
    job_df.to_pickle("job_df.pkl")
    print("DataFrame saved successfully as job_df.pkl.")


# Step 1: Load the Resume

def load_resume(file_path):
    if file_path.endswith('.pdf'):
        with open(file_path, 'rb') as file:
            reader = PdfReader(file)
            text = "".join([page.extract_text() for page in reader.pages ])
    elif file_path.endswith('.docx'):
        import docx2txt
        text = docx2txt.process(file_path)
    else:
        raise ValueError("Unsupported file format. Use PDF or DOCX.")
    return text


DataFrame loaded from pickle file.


In [ ]:
!pip install --upgrade openai
!pip install --upgrade pydantic


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [ ]:
# Step 4: Vectorization and Similarity Calculation (Batch PRocessing)

# def calculate_similarity(resume_text, job_descriptions):
#     vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
#     combined_texts = [resume_text] + job_descriptions
#     tfidf_matrix = vectorizer.fit_transform(combined_texts)
#     similarity_matrix = cosine_similarity(tfidf_matrix)
#     return similarity_matrix[0, 1:]


def calculate_similarity_in_batches(resume_text, job_descriptions, batch_size=5000):
    vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
    similarities = []

    for i in range(0, len(job_descriptions), batch_size):
        batch = job_descriptions[i:i + batch_size]
        combined_texts = [resume_text] + batch
        tfidf_matrix = vectorizer.fit_transform(combined_texts)
        batch_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()
        similarities.extend(batch_similarities)

    return similarities

In [ ]:
# Step 5: Match and Rank Jobs

# def match_jobs(resume_text, job_db):
#     job_db_subset = job_db.copy()
#     similarities = calculate_similarity(resume_text, job_db_subset['description'].tolist())
#     job_db_subset['similarity_score'] = similarities  # No normalization
#     sorted_jobs = job_db_subset.sort_values(by='similarity_score', ascending=False)
#     return sorted_jobs

def match_jobs(resume_text, job_db_path, chunksize=5000):
    top_matches = []
    for chunk in pd.read_csv(job_db_path, chunksize=chunksize):
        similarities = calculate_similarity_in_batches(resume_text, chunk['Resume'].tolist())
        chunk['similarity_score'] = similarities
        top_matches.append(chunk.sort_values(by='similarity_score', ascending=False).head(10))
    return pd.concat(top_matches).sort_values(by='similarity_score', ascending=False).head(10)

In [ ]:
import re

def extract_information_from_resume(resume_text):
    """
    Extract structured information from resume text.

    Parameters:
        resume_text (str): The input text of the resume.

    Returns:
        dict: Extracted information including Skills, Education, Work Experience, and Total Years of Experience.
    """
    # Initialize dictionary to store extracted info
    extracted_info = {
        "Skills": [],
        "Education": [],
        "Work Experience": [],
        "Total Years of Experience": ""
    }

    # Extract Skills
    skills_match = re.findall(r"Skills?:\s*(.*?)\n", resume_text, re.IGNORECASE | re.DOTALL)
    if skills_match:
        extracted_info["Skills"] = [skill.strip() for skill in skills_match[0].split(',')]

    # Extract Education or Qualifications
    education_match = re.findall(r"Education|Qualifications:?\s*(.*?)\n", resume_text, re.IGNORECASE)
    if education_match:
        extracted_info["Education"] = education_match

    # Extract Work Experience
    work_experience_match = re.findall(r"(Work Experience|Experience):?\s*(.*?)(?:\n\n|\Z)", resume_text, re.IGNORECASE | re.DOTALL)
    if work_experience_match:
        work_experience_details = work_experience_match[0][1]
        extracted_info["Work Experience"] = [exp.strip() for exp in work_experience_details.split('\n') if exp.strip()]

    # Extract Total Years of Experience
    years_match = re.findall(r"(\d+)\s*(years?|yrs?)\s*experience", resume_text, re.IGNORECASE)
    if years_match:
        extracted_info["Total Years of Experience"] = f"{years_match[0][0]} years"

    # Return the structured information
    return extracted_info


# Example usage
resume_text = """
John Doe
Skills: Python, Data Analysis, SQL, Machine Learning
Education: B.Sc. in Computer Science, XYZ University, 2020
Work Experience:
1. Data Scientist at ABC Corp (2021-2023)
2. Software Engineer at DEF Inc (2019-2020)
Total Years of Experience: 4 years
"""

extracted_info = extract_information_from_resume(resume_text)

print("Extracted Information:")
for key, value in extracted_info.items():
    print(f"{key}: {value}")


Extracted Information:
Skills: ['Python', 'Data Analysis', 'SQL', 'Machine Learning']
Education: ['']
Work Experience: ['1. Data Scientist at ABC Corp (2021-2023)', '2. Software Engineer at DEF Inc (2019-2020)', 'Total Years of Experience: 4 years']
Total Years of Experience: 


In [ ]:

# Main Execution
if __name__ == "__main__":
    # Load the resume
    resume_file_path = "./kaggle/input/resume.pdf"  # Replace with your Kaggle path
    if os.path.exists(resume_file_path):
        resume_text = load_resume(resume_file_path)
        print(resume_text)

        # Extract information from resume
        resume_info = extract_information_from_resume(resume_text)
        # display_extracted_info(resume_info)
        print(extracted_info)

        # Load job dataset
        job_dataset_path = "./input/Updated_Job_Dataset.csv"  # Replace with your dataset path
        if os.path.exists(job_dataset_path):
            job_df = pd.read_csv(job_dataset_path)

            # Match jobs and get recommendations
            recommended_jobs = match_jobs(resume_text, job_dataset_path)

            print("\nTop 10 Recommended Jobs:")
            #print(recommended_jobs[['title', 'description', 'published_date', 'country', 'similarity_score']].head(10))
            print(recommended_jobs[['Category', 'similarity_score']].head(10))
        else:
            print(f"Error: Job dataset file '{job_dataset_path}' not found.")
    else:
        print(f"Error: Resume file '{resume_file_path}' not found.")


Govind Sunilkumar
Aspiring Digital Electronics Engineer
♂phone+1-(945)2808460 /envel⌢pegovindpsunilkumar@gmail.com /linkedinGovind-Sunilkumar
Education
Masters of Science in Computer Engineering Aug 2023 - Dec 2024
Virginia Tech, Blacksburg CGPA: 4.0
B.Tech in Electronics and Communication Engineering Sep 2017 - May 2021
National Insititute of Technology, Karnataka CGPA: 7.35/10
Professional Experience
RTL Systems Design Intern, Skyworks Solutions, Austin May 2024 - Aug 2024
•Designed Skyworks proprietary communication protocol test driver on Zynq ZCU 102 FPGA , used Vivado flow to
generate bit stream and VBScript to test driver on PetaLinux Environment
•Executed Silicon Post-Validation tests on prototype test chip for Skyworks Solutions
Embedded Hardware Engineer, Bharat Electronics Limited, Bangalore Oct 2021 - Jun 2023
•Designed and Tested Microchip PIC microcontroller based Display System for aerospace applications using OrCAD
•Tested with I2C interface and developed Embedded C cod